<a href="https://colab.research.google.com/github/SatenikS/iphone12pro-availability/blob/main/iPhone12-Pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Scraper to check iPhone 12 Pro availability in Apple stores**


In this project I have created a scraper to identify iPhone 12 Pro availability in Apple stores for a given zip code.

At the time of creating this notebook the earliest delivery date for the new iPhone 12 Pro is 3-4 weeks away. There are also no phones available for in-store pickup. Customer representatives recommend checking the Apple website regularly for occasional phone availability. Few hours after creating this scraper I received an email notifying me of one available phone with my desired color and capacity. One day later I was holding the phone in my hand!

In [ ]:
# !pip install selenium
# !apt-get update # to update ubuntu to correctly run apt install
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
# !pip install random_user_agent

In [ ]:
# Imports libraries
import sys
import time
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem
import smtplib, ssl

In [ ]:
def check_iphone_availability(zip, size_max, color, gb):
  """Checks Apple website for iPhone pick-up availability

        Args:
            zip (str): Five digit zip code
            size_max (bool): True for iPhone 12 Pro Max
            color (str): Color of the iPhone (blue, silver, gray or gold)
            gb (int): GB size of the iPhone (128, 256 or 512)

        Returns:
            str: Response from Apple website
  """
  # Validates inputs
  if len(zip) != 5:
        raise ValueError('Invalid zip length, 5 digit string required')
  if color not in ['blue', 'silver', 'gray', 'gold']:
        raise ValueError('Invalid color, acceptable values are blue, silver, gray or gold')
  if gb not in [128, 256, 512]:
        raise ValueError('Invalid GB, acceptable values are 128, 256 or 512')

  # Sets sleep duration
  slp = 3

  # Initializes the webdriver
  software_names = [SoftwareName.CHROME.value]
  operating_systems = [OperatingSystem.WINDOWS.value,
                      OperatingSystem.LINUX.value]

  user_agent_rotator = UserAgent(software_names=software_names,
                                operating_systems=operating_systems,
                                limit=100)

  user_agent = user_agent_rotator.get_random_user_agent()

  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--window-size=1200,800')
  chrome_options.add_argument('--disable-gpu')
  chrome_options.add_argument(f'user-agent={user_agent}')
  wd = webdriver.Chrome('chromedriver',options=chrome_options)

  # Generates the URL
  colors = {'blue': 'pacific-blue',
            'gray': 'graphite',
            'silver': 'silver',
            'gold': 'gold'}
  url = 'https://www.apple.com/shop/buy-iphone/iphone-12-pro/'
  if size_max:
    url += '6.7'
  else:
    url += '6.1'
  url += '-inch-display-'
  url += str(gb) + 'gb-'
  url += colors[color]
  url += '-unlocked'

  # Navigates to the iPhone product page
  wd.get(url)

  # Sleeps
  time.sleep(slp)

  # Answers 'No' to the trade-in question
  element = wd.find_element_by_xpath('//*[@id="tradeup-inline-heroselector"]/div/div/fieldset/div/div[1]/div/div')
  element.click()

  # Sleeps
  time.sleep(slp)

  # Clicks 'Check availability' or 'Check another store'
  try:
    elements = wd.find_elements_by_xpath("//*[contains(text(), 'Check availability')]")
    elements[0].click()
  except:
    elements = wd.find_elements_by_xpath("//*[contains(text(), 'Check another store')]")
    elements[0].click()

  # Sleeps
  time.sleep(slp)

  # Inputs the zip code and presses the Return key
  element = wd.find_element_by_xpath('//*[@id="as-productlocator-query"]')
  element.clear()
  element.send_keys(zip)
  time.sleep(slp)
  element.send_keys(Keys.RETURN)

  # Sleeps
  time.sleep(slp)

  # Checks and returns the response
  try:
    # 'Not available today at 12 nearest stores' element
    element = wd.find_element_by_xpath('/html/body/overlay[14]/materializer/div/div/div/product-locator/div[2]/div[2]/div[1]/div[1]/div[2]/div[2]/button/span[1]')
  except:
    # 'Pick it up at an Apple Store near: #####' element
    element = wd.find_element_by_xpath('/html/body/overlay[14]/materializer/div/div/div/product-locator/div[2]/div[2]/div[1]/div[1]/div[2]/div[1]/h3')
  return element.text

In [ ]:
while True:
  # Checks availability
  response = check_iphone_availability(zip = '10001', size_max = True, color = 'blue', gb = 128)
  # Sends an email if the iPhone is available
  if response[:10] == 'Pick it up':
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    message = 'Subject: {}\n\n{}'.format('iPhone Alert!', response)
    # For Gmail login need to enable access for less secure apps
    server.login('from@gmail.com', 'password')
    server.sendmail('from@gmail.com', 'to@gmail.com', message)
    server.quit()  
    break
  else:
    # Sleeps for 5 minutes
    time.sleep(5 * 60)